# Linear Regression from the Ground Up

This notebook contains a direct implementation of linear regression using linear basis functions (polynomials).

## Preliminaries

The only libraries you should need for this notebook are `numpy`, `ipywidgets`, and `matplotlib`. You should be able to create the environment and run this notebook using comething like:

    conda create -n FML matplotlib ipywidgets
    conda activate FML
    jupyter notebook LinearRegressionMLE.ipynb

**Important**: If you are new to numerical programming in Python, I have also uploaded a VERY simple and VERY high-level tutorial of some of the numpy and Matplotlib features to the Moodle. There are many *excellent* tutorials out there.

## Basic Setup

We begin with a basic generative model (the one used in the introduction of the Bishop book). Scalar samples are generated from a sinusoidal function of the scalar input.

We begin by importing some useful stuff.

In [ ]:
# Standard imports for numerical programming.
import numpy as np
import matplotlib.pyplot as plt

Now we define the generative model in terms of a "true" function and an "observed" function corrupted with Gaussian noise with precision $\beta$. We also define a utility function to apply a function (passed as an argument) to a uniform random sample of inputs.

In [ ]:
# The true function -- that is, what we want to estimate from data.
def true_fn(x):
    return np.sin(2*np.pi*x)

# The observation process: true + Gaussian Noise.
def observed_fn(x, beta=1.0):
    return true_fn(x) + np.random.normal(0, np.sqrt(1/beta), size=x.shape)

# How we actually observe a function: uniformly generate N random
# xs in rng, apply the passed function to the xs.
def sample_from_fn(fn, N, rng=[0.0, 1.0]):
    xs = np.sort(np.random.random(size=(N,)) * (rng[1]-rng[0]) + rng[0])
    return (xs, fn(xs))

OK, let's play around with these functions. Visualization is our friend when trying to understand a problem from the ground up.

In [ ]:
# Sample 300 points from the true function and plot them (in green).
(true_xs, true_fxs) = sample_from_fn(true_fn, 300)
plt.plot(true_xs, true_fxs, 'g')

# Now sample 20 points from the observed function and plot
# the samples (in blue).
(observed_xs, observed_fxs) = sample_from_fn(lambda x: observed_fn(x, beta=20), 20)
plt.scatter(observed_xs, observed_fxs)

Note the **stochastic** nature of this process. Multiple executions of the sampling function will result in different realizations of the dataset.

## Fitting a Model to Data

We will now directly implement a *Regularized* least squares estimate of the optimal model parameters $\mathbf{w}^*$.

**Note**: this is not the Maximum Likelihood solution $\mathbf{w}_{\mbox{ML}}$ -- although if we select $\lambda$ correctly we can recover it.

In [ ]:
# This function evaluates the linear model with polynomial basis
# and parameters w on inputs xs.
def polynomial_model(xs, w):
    Phi = np.vstack([[x**i for i in range(len(w))] for x in xs])
    return np.squeeze(w[None, :] @ Phi.T)

# This the direct estimate of w using the Moore-Penrose psuedo-inverse.
# The order argument determines the order of the polynomial basis
# embedding, and lam is the regularization coefficient.
def regularized_least_squares(xs, ys, order, lam):
    Phi = np.vstack([[x**i for i in range(order+1)] for x in xs])
    w = np.linalg.inv(lam * np.eye(order+1) + Phi.T @ Phi) @ Phi.T @ ys
    return w

We can now solve for model parameters given the data in the current dataset (i.e. the last run of our generative process above). Let's solve for $\mathbf{w}^*$ for a specific $\lambda$ and order. Then we can plot the estimated solution (in red).

In [ ]:
# Estimate optimal model parameters.
w_star = regularized_least_squares(observed_xs, observed_fxs, 10, 0.002)

# Plot true function and observed samples.
plt.scatter(observed_xs, observed_fxs)
plt.plot(true_xs, true_fxs, 'g')

# And plot our polynomial estimate.
plt.plot(true_xs, polynomial_model(true_xs, w_star), 'r')
plt.ylim([-1.2, 1.2])

## Putting it All Together

Now we can implement a simple interactive widget that allows us to play with different polynomial orders and regularization coefficients. 

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# An interactive function that is called whenever the order or lambda
# (actually parameterized as log-lambda) changes.
@widgets.interact(order=(1, 100, 1), loglam=(-100.0, 0.0, 1.0))
def plotter(order, loglam):
    plt.figure(figsize=(13, 8))
    w_star = regularized_least_squares(observed_xs, observed_fxs, order, np.exp(loglam))
    plt.scatter(observed_xs, observed_fxs)
    plt.plot(true_xs, true_fxs, 'g')
    plt.plot(true_xs, polynomial_model(true_xs, w_star), 'r')
    plt.ylim([-1.2, 1.2])
